In [61]:
#Preparación de datos
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import sagemaker
# Modelado
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
#Prueba Inferencia.
import boto3
import pandas as pd


# Descarga y Lectura del dataset 

In [38]:
#%%sh
#wget -N -P data http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
#unzip data/bank.zip -d data

In [39]:
pwd

'/root/sagemaker-studio-demo'

In [41]:
DATA_PATH = '/root/sagemaker-studio-demo/bank/data/'
df = pd.read_csv(DATA_PATH+'bank-full.csv', sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [22]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [56]:
enc = OrdinalEncoder()
enc.fit(df)

data = enc.transform(df)

# Separamos el dataset en el grupo de entrenamiento y test

In [25]:
train, test = train_test_split(data, test_size=0.2)

In [27]:
train_file = DATA_PATH+'bank-train.csv'
pd.DataFrame.from_records(train).to_csv(train_file, index=False, header=True, sep=',')

test_file = DATA_PATH+'bank-test.csv'
pd.DataFrame.from_records(test).to_csv(test_file, index=False, header=True, sep=',')

# Subimos el dataset para entrenamiento al bucket de S3

In [30]:
session = sagemaker.Session()
uri = session.upload_data(path=train_file, key_prefix='bank')

Debemos tomar nota de la ruta de trabajo, que sera utilizada para generar el experimento en autopilot. 

In [29]:
print(uri)

s3://sagemaker-us-east-1-682428717171/bank/bank-train.csv


# Prueba de Inferencia.

In [62]:
sm = boto3.Session().client('runtime.sagemaker')

In [68]:
def infer(endpoint, input_data):
    response = sm.invoke_endpoint(EndpointName=endpoint, ContentType='text/csv', Accept='text/csv', Body=input_data)
    prediction = response['Body'].read().decode("utf-8")
    return prediction

In [118]:
endpoint = 'Amaliaconf251020Endpoint'
data = '17.0,1.0,1.0,1.0,0.0,916.0,1.0,0.0,0.0,8.0,5.0,279.0,0.0,0.0,0.0,3.0'

prediction = infer(endpoint, data)
print(prediction[0])


0


In [112]:
if prediction[0] == '0':
    print('No es apto para credito')
else:
    print('Es apto para credito')

No es apto para credito
